In [1]:
import texthero as hero
import pandas as pd
import fasttext
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [6]:
train_df = pd.read_csv('toxic_text_classification/train.csv/train.csv')
test_df = pd.read_csv('toxic_text_classification/test.csv/test.csv')
test_label_df = pd.read_csv('toxic_text_classification/test_labels.csv/test_labels.csv')

In [7]:
train_df.sample(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
115529,69d41fc56340e2ca,"English Book, English title. No american trans...",0,0,0,0,0,0
10098,1ab610d76b03b448,"""\n\n A little something for you... \n\n The...",0,0,0,0,0,0
120090,823517eeea74b48b,...And the statement is completely opinion. I...,0,0,0,0,0,0
145719,2129902e86ba7449,"""\n\n Vandalism \n\n """"However, due to the sw...",0,0,0,0,0,0
21995,39f9b712d14d82cc,and www.cornsilks.com sockpuppets,0,0,0,0,0,0
59247,9ebc771ca256b12c,Norlen \n\nIs she still VP?,0,0,0,0,0,0
154312,ac24cc08bc279c52,"""\nThank you for unblocking me. I feel a one m...",0,0,0,0,0,0
99612,14ee5b73253c55a0,""":Also, I don't want to slow you down on FA, b...",0,0,0,0,0,0
3579,099b672f19911fb2,wikipedian policy\n\nI see now how Wikipedia i...,1,0,1,0,1,0
129728,b5f2f4662d3d597d,Y HELLO THAR \n\nYou are one of the worst dele...,1,0,1,0,1,0


In [8]:
test_df.sample(10)

,id,comment_text
83258,8ad30675566f4be6,::We can change Canadian scholar to Harvard sc...
122245,cc2813667156e302,""" \n\n \n News Papers: \n\n \n Op-Eds: \n\n ..."
65077,6c43f26d964800ef,""" \n :::But I don't see why you seem to be tal..."
23071,267efe9c313c5f89,Please re-watch Outlaw Star. Suzuka did not di...
27000,2ced03e06937813b,haiasikag pooop onnn brandon mcdoggle berry ai...
131067,db0e6d84c80ff39f,""" \n\n == Memories Section == \n\n We should p..."
153116,ffeb7faf9662ed0f,"(UTC) \n\n ::FYI, I'm currently in edit confli..."
81974,88c567e1df05c78c,==Proposed move== \n Madonna (disambiguation) ...
141701,ece26c739c6ae0f7,""" \n\n == Spanish people tp Spaniards == \n\n ..."
19413,20948141a1ec0ed9,REDIRECT Talk:First Light (Preston book)


In [9]:
test_label_df.sample(10)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
30253,3257453fc5ee201a,-1,-1,-1,-1,-1,-1
99205,a589cf9433c2d816,-1,-1,-1,-1,-1,-1
130301,d9be258191e06344,0,0,0,0,0,0
102473,ab16243bc8223799,-1,-1,-1,-1,-1,-1
151334,fcea6ce95da5e8ea,-1,-1,-1,-1,-1,-1
117334,c3d960bdec6116e1,-1,-1,-1,-1,-1,-1
69613,73e51924905f2dcf,0,0,0,0,0,0
80434,863a5eb694a7865a,-1,-1,-1,-1,-1,-1
142711,ee8ec90b31f33407,-1,-1,-1,-1,-1,-1
138720,e7e1ba96376181ff,-1,-1,-1,-1,-1,-1


In [10]:
test_df = test_df.merge(test_label_df, left_on='id', right_on='id')
test_df = test_df[(test_df['toxic'] == 0) | (test_df['toxic']==1)]

In [11]:
train_df['comment_text_cleaned'] = train_df['comment_text'].pipe(hero.clean)
test_df['comment_text_cleaned'] = test_df['comment_text'].pipe(hero.clean)

C:\Users\Teng Li Yuan\AppData\Roaming\Python\Python37\site-packages\texthero\preprocessing.py:72: FutureWarning: The default value of regex will change from True to False in a future version.
  return input.str.replace(pattern, symbols)
C:\Users\Teng Li Yuan\AppData\Roaming\Python\Python37\site-packages\texthero\preprocessing.py:131: FutureWarning: The default value of regex will change from True to False in a future version.
  return input.str.replace(rf"([{string.punctuation}])+", symbol)


In [12]:
train_df= train_df[['toxic','comment_text_cleaned']]
test_df= test_df[['toxic','comment_text_cleaned']]

In [13]:
# Prefixing each row of the category column with '__label__'
train_df.iloc[:, 0] = train_df.iloc[:, 0].apply(lambda x: '__label__' + str(x))
test_df.iloc[:, 0] = test_df.iloc[:, 0].apply(lambda x: '__label__' + str(x))

In [14]:
# Prefixing each row of the category column with '__label__'
train_df[['toxic', 'comment_text_cleaned']].to_csv('toxic_train.txt', 
                                          index = False, 
                                          sep = ' ',
                                          header = None)

test_df[['toxic', 'comment_text_cleaned']].to_csv('toxic_test.txt', 
                                     index = False, 
                                     sep = ' ',
                                     header = None )

In [4]:
import time

In [35]:
start = time.time()
model = fasttext.train_supervised('toxic_train.txt', wordNgrams=3, epoch = 150, lr = 0.8)
end = time.time()
print(end - start)

227.50854539871216


In [36]:
model.test('toxic_test.txt' ,k=2) 

(63930, 0.5, 1.0)

In [37]:
y_pred = test_df.iloc[:, 1].apply(lambda x: model.predict(x)[0][0])
y_true = test_df.iloc[:, 0]

In [38]:
y_true

5         __label__0
7         __label__0
11        __label__0
13        __label__0
14        __label__0
             ...    
153058    __label__0
153059    __label__0
153062    __label__0
153063    __label__1
153064    __label__0
Name: toxic, Length: 63930, dtype: object

In [39]:
y_pred

5         __label__0
7         __label__0
11        __label__0
13        __label__0
14        __label__0
             ...    
153058    __label__0
153059    __label__1
153062    __label__0
153063    __label__1
153064    __label__0
Name: comment_text_cleaned, Length: 63930, dtype: object

In [40]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

  __label__0       0.97      0.95      0.96     57849
  __label__1       0.61      0.73      0.67      6081

    accuracy                           0.93     63930
   macro avg       0.79      0.84      0.81     63930
weighted avg       0.94      0.93      0.93     63930



In [41]:
confusion_matrix(y_true, y_pred)

array([[55025,  2824],
       [ 1631,  4450]], dtype=int64)

In [42]:
text = 'damn, I love it!'
model.predict()

In [73]:
testing_text = 'Damn, it is so funny'
clean_text = hero.clean(pd.Series(testing_text))[0]
model.predict(clean_text)

(('__label__1',), array([0.99981397]))

In [61]:
testing_text = 'Damn, I love it!'
clean_text = hero.clean(pd.Series(testing_text))[0]
model.predict(clean_text)

(('__label__0',), array([0.70555413]))